## Experiments
### Running finetuning using entire VGG model VS using precomputed conv. model outputs
### Conclusion: Precomputing the conv. model output seems fine... Final val_losss values 
### are very similar.

### Loading and Running the finetuning in plain old batches through the entire model

In [1]:
from Executor import *
# Reset the model, and run same finetuning using precomputed conv. model outputs. See if they match...
executor = ExecutorBuilder().\
    with_runID("first").\
    and_().\
    with_Vgg16().\
    and_().\
    train_batch_size(128). \
    and_(). \
    learn_rate(0.001).\
    and_().\
    data_on_path("../data/full/").\
    build()

Using Theano backend.
Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


Pretrained Vgg16 model loaded.
Found 6255 images belonging to 3 classes.
Found 1549 images belonging to 3 classes.
initialized training data from: ../data/full/train
initialized validation data from: ../data/full/valid
found number of softmax classes: 3


In [2]:
executor.precompute_conv_model_outputs()


precomputing conv. model outputs..
('models saved to files: ', 'precomputed_trn_features.first.h5', ' and ', 'precomputed_val_features.first.h5')
done.


<Executor.Executor instance at 0x7fce4b696d88>

### Performing same operation using precomputed conv. model outputs

In [3]:
executor.load_precomputed_conv_models()

loading precomputed conv. outputs...
done...


<Executor.Executor instance at 0x7fce4b696d88>

In [4]:
# Get the linear model 
linearModel = executor.get_rescaled_fc_model(new_dropout=executor.dropout)

# Set all layers except the last as being trainable
i=0
for layer in linearModel.layers:
    i+=1
    if(i!=7):
        layer.trainable=False
    if layer.trainable:
        print("found trainable layer: ",layer)


getting rescaled fc model...
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
done...
('updating dropout from: ', 0.5, ' to: ', 0.5)
('found trainable layer: ', <keras.layers.core.Dense object at 0x7fce41d5f550>)


In [5]:
# try fitting the model. Used an extremely small learning rate... still no avail
linearModel.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics = ['accuracy'])
linearModel.fit(executor.train_precomputed, executor.train_labels, batch_size=executor.batch_size,
                nb_epoch= 5, validation_data=(executor.val_precomputed, executor.val_labels))

Train on 6255 samples, validate on 1549 samples
Epoch 1/5
6255/6255 [==============================] - 3s - loss: 1.7486 - acc: 0.4480 - val_loss: 1.0942 - val_acc: 0.5339
Epoch 2/5
6255/6255 [==============================] - 3s - loss: 1.4125 - acc: 0.4854 - val_loss: 0.9470 - val_acc: 0.5449
Epoch 3/5
6255/6255 [==============================] - 3s - loss: 1.2658 - acc: 0.5058 - val_loss: 0.9190 - val_acc: 0.5565
Epoch 4/5
6255/6255 [==============================] - 3s - loss: 1.1813 - acc: 0.5132 - val_loss: 0.9132 - val_acc: 0.5481
Epoch 5/5
6255/6255 [==============================] - 3s - loss: 1.0951 - acc: 0.5303 - val_loss: 0.8989 - val_acc: 0.5668


In [6]:
linearModel.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics = ['accuracy'])
linearModel.fit(executor.train_precomputed, executor.train_labels, batch_size=executor.batch_size,
                nb_epoch= 5, validation_data=(executor.val_precomputed, executor.val_labels))

Train on 6255 samples, validate on 1549 samples
Epoch 1/5
6255/6255 [==============================] - 3s - loss: 1.0252 - acc: 0.5450 - val_loss: 0.8766 - val_acc: 0.5681
Epoch 2/5
6255/6255 [==============================] - 3s - loss: 0.9762 - acc: 0.5683 - val_loss: 0.8699 - val_acc: 0.5746
Epoch 3/5
6255/6255 [==============================] - 3s - loss: 0.9849 - acc: 0.5605 - val_loss: 0.8743 - val_acc: 0.5726
Epoch 4/5
6255/6255 [==============================] - 3s - loss: 0.9543 - acc: 0.5655 - val_loss: 0.8623 - val_acc: 0.5771
Epoch 5/5
6255/6255 [==============================] - 3s - loss: 0.9598 - acc: 0.5600 - val_loss: 0.8623 - val_acc: 0.5733


In [7]:
linearModel.compile(optimizer=Adam(lr=0.00001), loss='categorical_crossentropy', metrics = ['accuracy'])
linearModel.fit(executor.train_precomputed, executor.train_labels, batch_size=executor.batch_size,
                nb_epoch= 5, validation_data=(executor.val_precomputed, executor.val_labels))

Train on 6255 samples, validate on 1549 samples
Epoch 1/5
6255/6255 [==============================] - 3s - loss: 0.9604 - acc: 0.5635 - val_loss: 0.8590 - val_acc: 0.5817
Epoch 2/5
6255/6255 [==============================] - 3s - loss: 0.9459 - acc: 0.5691 - val_loss: 0.8585 - val_acc: 0.5771
Epoch 3/5
6255/6255 [==============================] - 3s - loss: 0.9463 - acc: 0.5677 - val_loss: 0.8579 - val_acc: 0.5765
Epoch 4/5
6255/6255 [==============================] - 3s - loss: 0.9501 - acc: 0.5597 - val_loss: 0.8582 - val_acc: 0.5759
Epoch 5/5
6255/6255 [==============================] - 3s - loss: 0.9536 - acc: 0.5703 - val_loss: 0.8574 - val_acc: 0.5771


In [8]:
linearModel.compile(optimizer=Adam(lr=0.000001), loss='categorical_crossentropy', metrics = ['accuracy'])
linearModel.fit(executor.train_precomputed, executor.train_labels, batch_size=executor.batch_size,
                nb_epoch= 5, validation_data=(executor.val_precomputed, executor.val_labels))

Train on 6255 samples, validate on 1549 samples
Epoch 1/5
6255/6255 [==============================] - 3s - loss: 0.9357 - acc: 0.5727 - val_loss: 0.8574 - val_acc: 0.5765
Epoch 2/5
6255/6255 [==============================] - 3s - loss: 0.9468 - acc: 0.5682 - val_loss: 0.8574 - val_acc: 0.5771
Epoch 3/5
6255/6255 [==============================] - 3s - loss: 0.9582 - acc: 0.5696 - val_loss: 0.8574 - val_acc: 0.5771
Epoch 4/5
6255/6255 [==============================] - 3s - loss: 0.9568 - acc: 0.5656 - val_loss: 0.8574 - val_acc: 0.5771
Epoch 5/5
6255/6255 [==============================] - 3s - loss: 0.9570 - acc: 0.5583 - val_loss: 0.8574 - val_acc: 0.5771


In [9]:
executor.init_vgg_with_retrained_fc_layers(linearModel)

<Executor.Executor instance at 0x7fce4b696d88>